# Text Question Answering scenario with RAI Dashboard

The [Stanford Question Answering Dataset (SQuAD)](https://huggingface.co/datasets/squad) is a reading comprehension dataset on a set of Wikipeda articles.  This notebook examines a huggingface question answering model evaluated on the dataset.

Install datasets to retrieve this dataset from huggingface:

In [ ]:
%pip install datasets
%pip install "pandas<2.0.0"

First, we need to specify the version of the RAI components which are available in the workspace. This was specified when the components were uploaded.

In [ ]:
version_string = "0.0.13"

We also need to give the name of the compute cluster we want to use in AzureML. Later in this notebook, we will create it if it does not already exist:

In [ ]:
compute_name = "cpucluster"

Finally, we need to specify a version for the data and components we will create while running this notebook. This should be unique for the workspace, but the specific value doesn't matter:

In [ ]:
rai_example_version_string = "6"

## Accessing the Data

We supply the data as a pair of parquet files and accompanying `MLTable` file. We can download them, preprocess them, and take a brief look:

In [ ]:
import os
import datasets
import pandas as pd

from sklearn import preprocessing

NUM_TEST_SAMPLES = 30

In [ ]:
def load_dataset(split):
    dataset = datasets.load_dataset("squad", split=split)
    questions = []
    context = []
    answers = []
    for row in dataset:
        context.append(row["context"])
        questions.append(row["question"])
        answers.append(row["answers"]["text"][0])
    return pd.DataFrame(
        {"context": context, "questions": questions, "answers": answers}
    )


pd_test_data = load_dataset("train")

test_data = pd_test_data[:NUM_TEST_SAMPLES]

In [ ]:
test_data

Now create the mltable:

In [ ]:
pq_filename = "hf_data.parquet"


def create_ml_table_file_contents(pq_filename):
    return (
        "$schema: http://azureml/sdk-2-0/MLTable.json\n"
        "type: mltable\n"
        "paths:\n"
        " - file: ./{0}\n"
        "transformations:\n"
        " - read_parquet\n"
    ).format(pq_filename)


def write_to_parquet(data, path, pq_filename):
    os.makedirs(path, exist_ok=True)
    data.to_parquet(os.path.join(path, pq_filename), index=False)


def create_ml_table_file(path, contents):
    with open(os.path.join(path, "MLTable"), "w") as f:
        f.write(contents)


test_data_path = "test_data"

write_to_parquet(test_data, test_data_path, pq_filename)

mltable_file_contents = create_ml_table_file_contents(pq_filename)
create_ml_table_file(test_data_path, mltable_file_contents)

Load some data for a quick view:

In [ ]:
import mltable

tbl = mltable.load(test_data_path)
test_df: pd.DataFrame = tbl.to_pandas_dataframe()

display(test_df)

The label column contains the answers:

In [ ]:
target_column_name = "answers"

First, we need to upload the datasets to our workspace. We start by creating an `MLClient` for interactions with AzureML:

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    ml_client = MLClient(
        credential=credential,
        subscription_id=subscription_id,
        resource_group_name=resource_group,
        workspace_name=workspace,
    )
except Exception:
    # If in compute instance we can get the config automatically
    from azureml.core import Workspace

    workspace = Workspace.from_config()
    workspace.write_config()
    ml_client = MLClient.from_config(
        credential=DefaultAzureCredential(exclude_shared_token_cache_credential=True),
        logging_enable=True,
    )

print(ml_client)

We can now upload the data to AzureML:

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_test_data = "Squad_Test_MLTable"

try:
    test_data = ml_client.data.get(
        name=input_test_data,
        version=rai_example_version_string,
    )
except Exception:
    test_data = Data(
        path=test_data_path,
        type=AssetTypes.MLTABLE,
        description="RAI Squad test data",
        name=input_test_data,
        version=rai_example_version_string,
    )
    ml_client.data.create_or_update(test_data)

# Creating the Model

To simplify the model creation process, we're going to use a pipeline.

We create a directory for the training script:

In [ ]:
import os

os.makedirs("squad_component_src", exist_ok=True)

Next, we write out our script to retrieve the huggingface question answering model:

In [ ]:
%%writefile squad_component_src/training_script.py

import argparse
import logging
import json
import os
import time

import mlflow
import mlflow.pyfunc

from transformers import pipeline

from azureml.core import Run

from azureml.rai.utils import PyfuncModel


_logger = logging.getLogger(__file__)
logging.basicConfig(level=logging.INFO)

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument(
        "--model_output_path", type=str, help="Path to write model info JSON"
    )
    parser.add_argument(
        "--model_base_name", type=str, help="Name of the registered model"
    )
    parser.add_argument(
        "--model_name_suffix", type=int, help="Set negative to use epoch_secs"
    )

    # parse args
    args = parser.parse_args()

    # return args
    return args


def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    _logger.info("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)

    # load the question-answering model
    pmodel = pipeline('question-answering')

    if args.model_name_suffix < 0:
        suffix = int(time.time())
    else:
        suffix = args.model_name_suffix
    registered_name = "{0}_{1}".format(args.model_base_name, suffix)
    _logger.info(f"Registering model as {registered_name}")

    my_mlflow = PyfuncModel(pmodel)

    # Saving model with mlflow
    _logger.info("Saving with mlflow")
    mlflow.pyfunc.log_model(
        python_model=my_mlflow,
        registered_model_name=registered_name,
        artifact_path=registered_name,
    )

    _logger.info("Writing JSON")
    dict = {"id": "{0}:1".format(registered_name)}
    output_path = os.path.join(args.model_output_path, "model_info.json")
    with open(output_path, "w") as of:
        json.dump(dict, fp=of)


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Now, we can build this into an AzureML component:

In [ ]:
from azure.ai.ml import load_component

yaml_contents = f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: rai_training_component
display_name: Squad training component for RAI example
version: {rai_example_version_string}
type: command
inputs:
  model_base_name:
    type: string
  model_name_suffix: # Set negative to use epoch_secs
    type: integer
    default: -1
outputs:
  model_output_path:
    type: path
code: ./squad_component_src/
environment: azureml://registries/azureml/environments/responsibleai-text-ubuntu20.04-py38-cpu/versions/39
command: >-
  python training_script.py
  --model_base_name ${{{{inputs.model_base_name}}}}
  --model_name_suffix ${{{{inputs.model_name_suffix}}}}
  --model_output_path ${{{{outputs.model_output_path}}}}
"""

yaml_filename = "SquadTextTrainingComp.yaml"

with open(yaml_filename, "w") as f:
    f.write(yaml_contents)

train_component_definition = load_component(source=yaml_filename)

ml_client.components.create_or_update(train_component_definition)

We need a compute target on which to run our jobs. The following checks whether the compute specified above is present; if not, then the compute target is created.

In [ ]:
from azure.ai.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if compute_name in all_compute_names:
    print(f"Found existing compute: {compute_name}")
else:
    my_compute = AmlCompute(
        name=compute_name,
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600,
    )
    ml_client.compute.begin_create_or_update(my_compute)
    print("Initiated compute creation")

## Running a training pipeline

Now that we have our training component, we can run it. We begin by generating a unique name for the mode;

In [ ]:
import time

model_base_name = "hf_qa_model"
model_name_suffix = "12486"
device = -1

Next, we define our training pipeline. This has two components. The first is the training component which we defined above. The second is a component to register the model in AzureML:

In [ ]:
from azure.ai.ml import dsl, Input

train_model_component = ml_client.components.get(
    name="rai_training_component", version=rai_example_version_string
)


@dsl.pipeline(
    compute=compute_name,
    description="Register Model for RAI Squad example",
    experiment_name=f"RAI_Squad_Example_Model_Training_{model_name_suffix}",
)
def my_training_pipeline(model_base_name, model_name_suffix, device):
    trained_model = train_component_definition(
        model_base_name=model_base_name,
        model_name_suffix=model_name_suffix,
    )
    trained_model.set_limits(timeout=3600)

    return {}


model_registration_pipeline_job = my_training_pipeline(
    model_base_name, model_name_suffix, device
)

With the training pipeline defined, we can submit it for execution in AzureML. We define a helper function to wait for the job to complete:

In [ ]:
from azure.ai.ml.entities import PipelineJob


def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

## Creating the RAI Text Insights

Now that we have our model, we can generate RAI Text insights for it. We will need the `id` of the registered model, which will be as follows:

In [ ]:
expected_model_id = f"{model_base_name}_{model_name_suffix}:1"
azureml_model_id = f"azureml:{expected_model_id}"

Next, we load the RAI components, so that we can construct a pipeline:

In [ ]:
squad_test_mltable = Input(
    type="mltable",
    path=f"{input_test_data}:{rai_example_version_string}",
    mode="download",
)

registry_name = "azureml"
credential = DefaultAzureCredential()

ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name,
)

rai_text_insights_component = ml_client_registry.components.get(
    name="rai_text_insights", version=version_string
)

We can now specify our pipeline. Complex objects (such as lists of column names) have to be converted to JSON strings before being passed to the components.

In [ ]:
import json
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes


@dsl.pipeline(
    compute=compute_name,
    description="Example RAI computation on Squad data",
    experiment_name=f"RAI_Squad_Example_RAIInsights_Computation_{model_name_suffix}",
)
def rai_squad_question_answering_pipeline(
    target_column_name,
    test_data,
    classes,
    use_model_dependency,
):
    # Initiate the RAIInsights
    rai_text_job = rai_text_insights_component(
        task_type="question_answering",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        test_dataset=test_data,
        target_column_name=target_column_name,
        classes=classes,
        use_model_dependency=use_model_dependency,
    )
    rai_text_job.set_limits(timeout=7200)

    rai_text_job.outputs.dashboard.mode = "upload"
    rai_text_job.outputs.ux_json.mode = "upload"

    return {
        "dashboard": rai_text_job.outputs.dashboard,
        "ux_json": rai_text_job.outputs.ux_json,
    }

Next, we define the pipeline object itself, and ensure that the outputs will be available for download:

In [ ]:
import uuid
from azure.ai.ml import Output

insights_pipeline_job = rai_squad_question_answering_pipeline(
    target_column_name=target_column_name,
    test_data=squad_test_mltable,
    use_model_dependency=True,
    classes="[]",
)

rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)
insights_pipeline_job.outputs.ux_json = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/ux_json/",
    mode="upload",
    type="uri_folder",
)

And submit the pipeline to AzureML for execution:

In [ ]:
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

The dashboard should appear in the AzureML portal in the registered model view. The following cell computes the expected URI:

In [ ]:
sub_id = ml_client._operation_scope.subscription_id
rg_name = ml_client._operation_scope.resource_group_name
ws_name = ml_client.workspace_name

expected_uri = f"https://ml.azure.com/model/{expected_model_id}/model_analysis?wsid=/subscriptions/{sub_id}/resourcegroups/{rg_name}/workspaces/{ws_name}"

print(f"Please visit {expected_uri} to see your analysis")

## Constructing the pipeline in YAML

It is also possible to specify the pipeline as a YAML file, and submit that using the command line. We will now create a YAML specification of the above pipeline and submit that:

In [ ]:
yaml_contents = f"""
$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
experiment_name: RAI_Squad_Example_RAIInsights_Computation_{rai_example_version_string}
type: pipeline

compute: azureml:cpucluster

inputs:
  registered_model_name: hf_qa_model
  hf_model_info: {expected_model_id}
  my_test_data:
    type: mltable
    path: azureml:{input_test_data}:{rai_example_version_string}
    mode: download

settings:
  default_datastore: azureml:workspaceblobstore
  default_compute: azureml:cpucluster
  continue_on_step_failure: false

jobs:
  analyse_model:
    type: command
    component: azureml://registries/azureml/components/rai_text_insights/versions/{version_string}
    inputs:
      task_type: question_answering
      model_input:
        type: mlflow_model
        path: {azureml_model_id}
      model_info: ${{{{parent.inputs.hf_model_info}}}}
      test_dataset: ${{{{parent.inputs.my_test_data}}}}
      target_column_name: {target_column_name}
      maximum_rows_for_test_dataset: 5000
      classes: '[]'
      enable_explanation: True
      enable_error_analysis: True
"""

yaml_pipeline_filename = "rai_text_example.yaml"

with open(yaml_pipeline_filename, "w") as f:
    f.write(yaml_contents)

The created file can then be submitted using the Azure CLI:

In [ ]:
cmd_line = [
    "az",
    "ml",
    "job",
    "create",
    "--resource-group",
    rg_name,
    "--workspace",
    ws_name,
    "--file",
    yaml_pipeline_filename,
]

import subprocess

try:
    cmd = subprocess.run(cmd_line, check=True, shell=True, capture_output=True)
except subprocess.CalledProcessError as cpe:
    print(f"Error invoking: {cpe.args}")
    print(cpe.stdout)
    print(cpe.stderr)
    raise
else:
    print("Azure CLI submission completed")